In [ ]:
""" Code demonstrating how to fit hopping integrals to inverse-SK results for a one-element system """

In [ ]:
import contextlib
import itertools as it
import functools
import math
import os
import pathlib

from types import SimpleNamespace

import numpy as np

import matplotlib.pyplot as plt

import plato_pylib.shared.ucell_class as UCell
import plato_pylib.plato.mod_plato_inp_files as modInp
import plato_pylib.plato.parse_inv_sk as parseInvSk
import plato_pylib.plato.parse_tbint_files as parseTbint
import plato_pylib.utils.job_running_functs as jobRun


import plato_fit_integrals.core.coeffs_to_tables as coeffsToTables
import plato_fit_integrals.core.obj_funct_calculator as objFunctCalc
import plato_fit_integrals.core.create_analytical_reprs as analyticFuncts
import plato_fit_integrals.core.workflow_coordinator as wFlow
import plato_fit_integrals.core.opt_runner as optRun

import plato_fit_integrals.initialise.create_coeff_tables_converters as createCoeffTables
import plato_fit_integrals.initialise.fit_analytic_to_initial_tables as fitInit
import plato_fit_integrals.initialise.obj_functs_targ_vals as createObjFuncts

MODEL_DATAFOLDER = "Test/format_4"
FULL_PATH_MODEL_DATAFOLDER = modInp.getAbsolutePathForPlatoTightBindingDataSet(MODEL_DATAFOLDER)

ATOM_SYMBOL = "Si"
N_CORES = 5

WORK_FOLDER = os.path.abspath("work_folder")

RUN_REF_JOBS = False #Set to false to save time if you've already run the reference inverse-SK jobs


#We only fit a single set of hopping integrals to inv-sk here, these define the table we do
SHELL_A = 1 #Starts from 0 i think
SHELL_B = 1
ORB_ANG_MOM = 2 #Starts from 1; 2 is pi

ORB_ANG_MOM_TO_STR = {1:"sigma",2:"pi",3:"delta"}

INTEG_STR = "hopping"


#Analytical function parameters
RCUT_VAL = parseTbint.getBdtRcut( os.path.join(FULL_PATH_MODEL_DATAFOLDER, "{}_{}.bdt".format(ATOM_SYMBOL,ATOM_SYMBOL)) )
REF_R0 = 1.0
N_POLY = 3
TAIL_DELTA = 0.5


In [ ]:
def generateTrimerGeometries(atomSymbol):
    boxLength = 50 #We use fract coords, so changing this WILL change the Si-Si-Si distances
    lattVects = [ [boxLength, 0.0      , 0.0      ],
                  [0.0      , boxLength, 0.0      ],
                  [0.0      , 0.0      , boxLength] ]
    
    allFractCoords = _getFractCoords(atomSymbol)
    outGeoms = list()
    for x in allFractCoords:
        outGeoms.append( UCell.UnitCell.fromLattVects(lattVects, fractCoords=x)  )

    return outGeoms

#I generated these with code from three_body_e0; which is currently one of my unfinished/abandoned github repos
def _getFractCoords(atomSymbol):
    fractCoordsA = [[0.5, 0.5, 0.46],
                [0.5, 0.5, 0.54],
                [0.5, 0.5992156741649222, 0.5275]]
    
    fractCoordsB = [[0.5, 0.5, 0.48],
                    [0.5, 0.5, 0.52],
                    [0.5, 0.5759934207678533, 0.545]]
    
    fractCoordsC = [[0.5, 0.5, 0.46],
                    [0.5, 0.5, 0.54],
                    [0.5, 0.56, 0.54]]
    
    fractCoordsD = [[0.5, 0.5, 0.44],
                    [0.5, 0.5, 0.56],
                    [0.5, 0.575424723326565, 0.5333333333333333]]

    fractCoordsE = [[0.5, 0.5, 0.46],
                    [0.5, 0.5, 0.54],
                    [0.5, 0.5692820323027551, 0.5]]
    allFractCoords = [fractCoordsA, fractCoordsB, fractCoordsC, fractCoordsD, fractCoordsE]

    for coordSet in allFractCoords:
        for atom in coordSet:
            atom.append(atomSymbol)
    return allFractCoords


In [ ]:
class InvSKTrimerCalculation:
    
    def __init__(self,jobName, workFolder, geom:"UCell obj"):
        self.jobName = os.path.splitext(jobName)[0]
        self.workFolder = os.path.abspath(workFolder)
        self.geom = geom
    
    @property
    def inpPath(self):
        return os.path.join( self.workFolder, self.jobName + ".in" )

    @property
    def invSkPath(self):
        return os.path.join(self.workFolder, self.jobName + "_Si_Si_SK.csv")
    
    def writeFile(self):
        startDict = _getOptDictBasicInvSkFile()
        strDict = modInp.getStrDictFromOptDict(startDict,"dft2")
        geomSection = modInp.getPlatoGeomDictFromUnitCell(self.geom)
        strDict.update(geomSection)
        pathlib.Path(self.workFolder).mkdir(exist_ok=True,parents=True)
        modInp.writePlatoOutFileFromDict(self.inpPath,strDict)
    
        
#Note that inv-SK on is the default
def _getOptDictBasicInvSkFile():
    outOptDict = {k.lower():v for k,v in modInp.getDefOptDict("dft2").items()}
    outOptDict["IntegralMeshSpacing".lower()] = [40,40,40]
    outOptDict["dataset"] = MODEL_DATAFOLDER
    return outOptDict


def parseAllRefInvSkData(trimerCalcObjs:list):
    parsedData = [ parseInvSk.parseInvSK(x.invSkPath) for x in trimerCalcObjs ]
    for x in parsedData[1:]:
        parsedData[0].addInvSKParsedFileData(x)
    return parsedData[0]
    
    

In [ ]:

class InvSkWorkflow(wFlow.WorkFlowBase):
    
    def __init__(self, invSkData, getIntegralsFunction):
        self.invSkData = invSkData
        self.integGetter = getIntegralsFunction

    @property
    def namespaceAttrs(self):
        return ["rmsd"]

    @property
    def workFolder(self):
        return None

    def run(self):
        #Get rmsd value
        newInts = self.integGetter().integrals
        sqrDiffs = list()
        for row in self.invSkData:
            xVal,yVal = row[0],row[1]
            calcYVal = fitInit.getInterpYValGivenXValandInpData(xVal,newInts)
            sqrDiffs.append(  (yVal-calcYVal)**2 )
        sumSqrDev = sum(sqrDiffs)
        rmsd = math.sqrt( sumSqrDev/len(sqrDiffs) )
        
        #Put it in a namespace
        self.output = SimpleNamespace(rmsd=rmsd)



In [ ]:
#Step 1 is to create files for inverse-SK calculations
allRefGeoms = generateTrimerGeometries(ATOM_SYMBOL)
refNames = ["refCalc_{}".format(x) for x in range(len(allRefGeoms))]

#Create objects
refCalcObjs = list()
for jName,geom in it.zip_longest(refNames,allRefGeoms):
    refCalcObjs.append( InvSKTrimerCalculation( jName,WORK_FOLDER,geom )  )

    
#Write files + get the file paths
[x.writeFile() for x in refCalcObjs]
allInvSkFilePaths = [x.inpPath for x in refCalcObjs]

In [ ]:
#Step 2 - Run all the inverse-SK reference jobs if needed
if RUN_REF_JOBS:
    jobRun.runInvSkParralel(allInvSkFilePaths,N_CORES)


In [ ]:
#Step 3 - setup coeffs to tables [which gets us access to initial integral tables too]
integHolder = createCoeffTables.createIntegHolderFromModelFolderPath(FULL_PATH_MODEL_DATAFOLDER)
integInfo = coeffsToTables.IntegralTableInfo(FULL_PATH_MODEL_DATAFOLDER, INTEG_STR, ATOM_SYMBOL, ATOM_SYMBOL,
                                            SHELL_A, SHELL_B, ORB_ANG_MOM)
startTable = integHolder.getIntegTable(INTEG_STR, ATOM_SYMBOL, ATOM_SYMBOL, SHELL_A, SHELL_B, ORB_ANG_MOM,
                                      inclCorrs=False)

hopAnalyticalFunction = analyticFuncts.Cawkwell17ModTailRepr(rCut=RCUT_VAL, refR0=REF_R0, valAtR0=1,
                                                       tailDelta=TAIL_DELTA, nPoly=N_POLY)
hopAnalyticalFunction.valAtR0 = fitInit.getInterpYValGivenXValandInpData(REF_R0, startTable.integrals)

coeffsTablesConverter = coeffsToTables.CoeffsTablesConverter([hopAnalyticalFunction], [integInfo], integHolder)


In [ ]:
#Step 4 - set the workflows up

#Need the inverse SK data for this
allSkData = parseAllRefInvSkData(refCalcObjs)
allSkData.removeXtalFieldTerms()
valType = "hVal"
relevantSkData = allSkData.getAllValsOrbPair(valType, SHELL_A, SHELL_B, bondType=ORB_ANG_MOM_TO_STR[ORB_ANG_MOM])
relevantSkData = np.array(relevantSkData)

#Also need a getter to get us the updated integrals
integGetter = functools.partial(integHolder.getIntegTable,INTEG_STR, ATOM_SYMBOL, ATOM_SYMBOL, SHELL_A, SHELL_B, ORB_ANG_MOM)
invSkWorkFlow = InvSkWorkflow(relevantSkData, integGetter)

workflowCoordinator = wFlow.WorkFlowCoordinator([invSkWorkFlow])


In [ ]:
#STEP 5 - Create an, effective blank, objective function calculator
blankObjFunct = createObjFuncts.createSimpleTargValObjFunction("blank")
targValuesWithObjFuncts = SimpleNamespace(rmsd=(0,blankObjFunct))
objFunctCalculator = objFunctCalc.ObjectiveFunctionContrib(targValuesWithObjFuncts)

In [ ]:
#Step 6 - plot the initial inverse-SK vs normal
figA = plt.figure()
ax1 = figA.add_subplot(1,1,1)
ax1.plot(startTable.integrals[:,0],startTable.integrals[:,1])
ax1.scatter(relevantSkData[:,0], relevantSkData[:,1])


In [ ]:
#Step 7 - Create Total objective function
totalObjFunction = optRun.ObjectiveFunction(coeffsTablesConverter, workflowCoordinator, objFunctCalculator)

In [ ]:
#Step 8 - Run the fit
fitRes = optRun.carryOutOptimisationBasicOptions(totalObjFunction)

In [ ]:
#Step 9 - plot new integrals vs reference data
newIntVals = integHolder.getIntegTable(INTEG_STR, ATOM_SYMBOL, ATOM_SYMBOL, SHELL_A, SHELL_B, ORB_ANG_MOM,
                                      inclCorrs=True)

figB = plt.figure()
axB = figB.add_subplot(1,1,1)
axB.plot(newIntVals.integrals[:,0], newIntVals.integrals[:,1])
axB.scatter(relevantSkData[:,0], relevantSkData[:,1])
